# Understanding Lego Sets Popularity

## Background

We recently applied to work as a data analyst at the Lego Group. As part of the job interview process, we received the following take-home assignment:

We are asked to use the provided dataset to understand the popularity of different Lego sets and themes. The idea is to become familiarized with the data to be ready for an interview with a business stakeholder.

### Research questions

Create a report to summarize our findings for the following questions. The following questions need to be answered:

1. What is the average number of Lego sets released per year?
2. What is the average number of Lego parts per year?
3. Create a visualization for item 2.
4. What are the 5 most popular colors used in Lego parts?
5. [Optional] What proportion of Lego parts are transparent?
6. [Optional] What are the 5 rarest lego bricks?
7. Summarize findings.

## Database Schema

Visualization of how the tables are related to each other. ([source](https://rebrickable.com/downloads)):

![erd](data/lego_erd.png)

## Database Description

#### inventory_parts
- "inventory_id" - id of the inventory the part is in (as in the inventories table)
- "part_num" - unique id for the part (as in the parts table)
- "color_id" - id of the color
- "quantity" - the number of copies of the part included in the set
- "is_spare" - whether or not it is a spare part

#### parts
- "part_num" - unique id for the part (as in the inventory_parts table)
- "name" - name of the part
- "part_cat_id" - part category id (as in part_catagories table)

#### part_categories
- "id" - part category id (as in parts table)
- "name" - name of the category the part belongs to

#### colors
- "id" - id of the color (as in inventory_parts table)
- "name" - color name
- "rgb" - rgb code of the color
- "is_trans" - whether or not the part is transparent/translucent

#### inventories
- "id" - id of the inventory the part is in (as in the inventory_sets and inventory_parts tables)
- "version" - version number
- "set_num" - set number (as in sets table)

#### inventory_sets
- "inventory_id" - id of the inventory the part is in (as in the inventories table)
- "set_num" - set number (as in sets table)
- "quantity" - the quantity of sets included

#### sets
- "set_num" - unique set id (as in inventory_sets and inventories tables)
- "name" - the name of the set
- "year" - the year the set was published
- "theme_id" - the id of the theme the set belongs to (as in themes table)
- num-parts - the number of parts in the set

#### themes
- "id" - the id of the theme (as in the sets table)
- "name" - the name of the theme
- "parent_id" - the id of the larger theme, if there is one


***Acknowledgments**: Rebrickable.com*

## Methods 

The database has been querying with PostgreSQL using Common Table Expression, aggregate functions and filtering. 

The visualizations have been created using Python (plotly). 

# Analysis

## 1. What is the average number of Lego sets released per year?


**Answer:** The average number of Lego sets per year is 176 sets. 

**Solution:**
The dataset shows the list of all sets during the available period (that is 66 years from 1950 to 2017). Python code below do the calculations to the dataset to return the final result. 

In [68]:
--select all data sorting by year

SELECT set_num, year
FROM sets
ORDER BY year DESC

,set_num,year
0,71017-19,2017
1,71017-18,2017
2,71017-17,2017
3,71017-16,2017
4,71017-15,2017
...,...,...
11668,700.1.2-1,1950
11669,700.B.3-1,1950
11670,700.B.1-1,1950
11671,700.B.2-1,1950


In [69]:
# calculate the total sum of sets
sum_sets = df['set_num'].count()

# calculate the unique years
count_year = df['year'].nunique()

# calulate average number of sets per year
avg = round(sum_sets / count_year, 1)

print("The total amount of produced Lego sets:", sum_sets)
print("Number of years in the dataset:", count_year)
print("The average number of Lego sets per year:", avg)

The total amount of produced Lego sets: 11673
Number of years in the dataset: 66
The average number of Lego sets per year: 176.9


## 2. What is the average number of Lego parts per year?

### 2.1. Total average ### 
In first solution as average we take the total of the parts released over all of the years. With this logic, we have the result of 28698 parts/year as average. 

In [70]:
-- find average number of lego parts for all the year

-- CTE for all parts
WITH data_per_year AS (
    SELECT year, 
    SUM(num_parts) AS parts_per_year, 
    COUNT(num_parts) AS sets_per_year,
    SUM(num_parts) / COUNT(num_parts) AS avg_parts_per_set_per_year
FROM sets
GROUP BY year
ORDER BY year)

-- calculate the average parts per all of the years
SELECT ROUND(AVG(parts_per_year), 2) AS avg_parts
FROM data_per_year

,avg_parts
0,28698.32


### 2.2. Average per each year ### 
The average number of pieces across all sets has been determined to be 162. Additionally, by analyzing the data further, we are able to calculate the average number of pieces produced by the company each year, as demonstrated in the table below. 

In [71]:
-- find average of lego parts by each year

SELECT ROUND(AVG(num_parts), 0) AS avg_parts_general
FROM sets

,avg_parts_general
0,162


In [72]:
-- find average of lego parts by each year

SELECT year, 
	SUM(num_parts) AS sum_parts, 
	COUNT(num_parts) AS count_parts,
	ROUND(AVG(num_parts), 0) AS avg_parts_per_year
FROM sets
GROUP BY year
ORDER BY year 

,year,sum_parts,count_parts,avg_parts_per_year
0,1950,71,7,10
1,1953,66,4,17
2,1954,173,14,12
3,1955,1032,28,37
4,1956,222,12,19
...,...,...,...,...
61,2013,107537,593,181
62,2014,121007,713,170
63,2015,134110,665,202
64,2016,150834,596,253


## 3. Create a visualization for item 2.

The bar chart below built based on the data from item 2.2 (the table is copied below). ach bar represents average part released per each year. Green line on the chart represents average number across all sets. 

In [73]:
-- find average lego parts per each year

SELECT year, 
	ROUND(AVG(num_parts), 0) AS avg_parts
FROM sets
GROUP BY year
ORDER BY year 

,year,avg_parts
0,1950,10
1,1953,17
2,1954,12
3,1955,37
4,1956,19
...,...,...
61,2013,181
62,2014,170
63,2015,202
64,2016,253


In [74]:
# Import plotly module
import plotly.express as px

# Create bar chart trace
fig = px.bar(df2,
             x="year",
             y="avg_parts",
             title="<b>Average number of Lego parts per year</b>",
             labels={"year": "Year of Production", "avg_parts": "Average Number of Parts"})

# Add horizontal line at y=162
fig.add_shape(type='line',
              x0=df2['year'].min(), y0=162,
              x1=df2['year'].max(), y1=162,
              line=dict(color='yellow', width=2),
              xref='x', yref='y')

# Add title to the line at y=162
fig.add_annotation(x=df2['year'].min() + 2, y=172,
                   text="Average",
                   showarrow=False,
                   font=dict(color='gray', size=10),
                   xref='x',
                   yref='y')

# Show the plot
fig.show()

## 4. What are the 5 most popular colors used in Lego parts?

**Answer:** 5 the most popular colors used in Lego parts:
1) Black (115085 parts)
2) White (66536 parts)
3) Light Bluish Gray (55302 parts)
4) Red (50213 parts)
5) Dark Bluish Gray (43907 parts)

**Solution:** To calculate the 5 most popular colors in Lego parts, we joined 'colors' and 'inventory_parts' tables, and counted the amount of each color. 

In [75]:
-- find 5 most popular colors in lego parts

SELECT c.name AS color_name,
	COUNT(ip.part_num) AS count
FROM colors AS c
JOIN inventory_parts AS ip ON c.id=ip.color_id
GROUP BY color_name
ORDER BY count DESC
LIMIT 5

,color_name,count
0,Black,115085
1,White,66536
2,Light Bluish Gray,55302
3,Red,50213
4,Dark Bluish Gray,43907


In [76]:
# Create bar chart
fig = px.bar(df3,
             x="color_name",
             y="count",
             title="<b>5 most popular colors used in Lego parts</b>",
             labels={"color_name": "Color", "count": "Number of parts"})

# Show the plot
fig.show()

## 5. What proportion of Lego parts are transparent?

**Answer:** The proportion of transparent lego parts to all parts is around 6.3%

**Solution:** We need to find number of all parts and number of transparent parts, which are 580069 and 36318 retrospectively, that leads to result around 6.3% transparent lego parts. 

In [77]:
-- find proportion of transparent parts 

-- CTE for all parts
WITH all_parts AS (
	SELECT COUNT(DISTINCT part_num)::float AS all_parts
	FROM inventory_parts),

-- CTE for transparent parts
transparent_parts AS (
	SELECT COUNT(DISTINCT ip.part_num) AS transparent_parts
	FROM inventory_parts AS ip
	JOIN colors AS c ON ip.color_id = c.id
	WHERE is_trans = True)

-- calculate the proportion
SELECT (transparent_parts/all_parts) AS proportion_transparent_parts
FROM transparent_parts, all_parts

,proportion_transparent_parts
0,0.062949


## 6. What are the 5 rarest lego bricks?

**Answer:** 5 the most rarest Lego bricks:
1) Bricks Printed (4580 released parts)
2) Bricks Wedged (9209 released parts)
3) Technic Bricks (36173 released parts)
4) Bricks Curved (38560 released parts)
5) Bricks Round and Cones (48525 released parts)

**Solution:** To calculate the five rarest categories of bricks, we need to find the groups of bricks that have been released in the lowest amount.

In [78]:
-- find 5 rarest lego bricks

SELECT pc.name, 
	SUM(IP.quantity) AS total_usage
FROM part_categories AS pc
JOIN parts AS p ON pc.id = p.part_cat_id
JOIN inventory_parts AS ip ON p.part_num = ip.part_num
WHERE PC.name LIKE '%rick%'
GROUP BY PC.name
ORDER BY total_usage
LIMIT 5

,name,total_usage
0,Bricks Printed,4580
1,Bricks Wedged,9209
2,Technic Bricks,36173
3,Bricks Curved,38560
4,Bricks Round and Cones,48525


## Summary

### 1. What is the average number of Lego sets released per year?
In the total span of 66 years, 11673 sets have been released. This means an average of 176 sets per year have been released.

### 2. What is the average number of Lego parts over total time span and per year?
In the total span of 66 years, a grand total of 1894089 parts have been released, an average of 28698 parts per year have been released. We can see that in 1951 and 1952 no parts were released.

The average amount per year can also be calculated and visualized to detect a trend. Referring to the graph in item 3, we can can conclude there has been an increase in released parts throughout the years.

### 3. What are the 5 most popular colors used in Lego parts? 
The most popular colors used in Lego parts are:
1. Black (115085 parts)
2. White (66536 parts)
3. Light Bluish Gray (55302 parts)
4. Red (50213 parts)
5. Dark Bluish Gray (43907 parts)

### 4. What proportion of Lego parts are transparent?
About 6.3% of the Lego bricks are transparent.

### 5. What are the 5 rarest categories of Lego bricks?
The rarest categories of Lego bricks are:
1) Bricks Printed (4580 released parts)
2) Bricks Wedged (9209 released parts)
3) Technic Bricks (36173 released parts)
4) Bricks Curved (38560 released parts)
5) Bricks Round and Cones (48525 released parts)